In [91]:
import pickle
G = pickle.load(open('bf16a3a0-75de-4ac5-9ff2-e2f25edf4ba4/state_0.pkl', 'rb'))
print(G)

DiGraph with 143 nodes and 186 edges


In [93]:
# print all the unique edge types
edge_types = set()
for edge in G.edges(data=True):
    edge_types.add(edge[2]['type'])
print(edge_types)

{'imports_directly', 'function-call', 'imports_from', 'root-to-dir', 'is_a_submodule_of', 'EXTERNAL_DEPENDENCY', 'function', 'REQUIRED_TYPE', 'UNKNOWN'}


In [100]:
edges = G.edges(data=True)
for edge in edges:
    print((edge[0], edge[2]['type'], edge[1])) if edge[2]['type'] == 'function' else None

('embed.py', 'function', 'get_embedding')
('embed.py', 'function', 'open_data_json')
('embed.py', 'function', 'save_embeddings')
('embed.py', 'function', 'embed_pipeline')
('chunker.py', 'function', 'extract_education_and_experience')
('chunker.py', 'function', 'open_files')
('chunker.py', 'function', 'store_data')
('chunker.py', 'function', 'chunking_pipeline')
('chunker.py', 'function', 'final_chunks')
('chunker.py', 'function', 'split_files')
('update_database.py', 'function', 'template_upload_pipeline')
('agents.py', 'function', 'create_thread')
('agents.py', 'function', 'get_response')
('agents.py', 'function', 'create_assistant')
('agents.py', 'function', 'add_files')
('agents.py', 'function', 'get_run')
('agents.py', 'function', 'add_message')
('agents.py', 'function', 'run_assistant')
('search.py', 'function', 'open_data_json')
('search.py', 'function', 'cosine_similarity')
('search.py', 'function', 'open_embeddings_json')
('search.py', 'function', 'vector_search')
('dfs.py', '